In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, cross_val_score, validation_curve
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl

In [2]:
df_data = pd.read_csv('../data/train.csv')
df_data.head()

,id,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,1,Acer_Opalus,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
1,2,Pterocarya_Stenoptera,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
2,3,Quercus_Hartwissiana,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
3,5,Tilia_Tomentosa,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
4,6,Quercus_Variabilis,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


## Génération des données d'entrainement et de test

In [3]:
X = df_data.copy().drop(['id', 'species'],axis='columns').values

species = df_data['species'].unique()

# Création des vecteurs one-hot pour la matrice de targets
t = np.zeros((len(X),species.size)) 
for i in range(species.size):
    t[df_data['species'] == species[i], i] = 1

We split the data in two: 
- 70% D_train;
- 30% D_test;

In addition, we have in our dataset 99 different species each represented by 10 individuals.
If we apply a 70%/30% ration we have on average:
- 7 individuals per species for the training set.
- 3 individuals per species for the test set.

In [4]:
X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=0.3, random_state=0)

We obtain the following result:

In [5]:
print("size X_train: "+str(X_train.shape))
print("size t_train: "+str(t_train.shape))
print()
print("size X_test: "+str(X_test.shape))
print("size t_test: "+str(t_test.shape))

size X_train: (693, 192)
size t_train: (693, 99)

size X_test: (297, 192)
size t_test: (297, 99)


## Model declaration

In [8]:
model = MLPClassifier(random_state=1, max_iter=3000, hidden_layer_sizes=(96, 96), tol=1e-3)
model.fit(X_train, t_train)
print("accuracy of {:.2f}% for the training set".format(model.score(X_train, t_train)*100))

accuracy of 98.56% for the training set


In [10]:
print("accuracy of {:.2f}% for the test set".format(model.score(X_test, t_test)*100))

accuracy of 70.37% for the test set


The MLP with scikit-learn take a lot of time to converge which will make very difficult the optimization of the hyperparameters. To have more control with the MLP, we decide to switch with an implementation of the MLP with PyTorch. Thus, allowing us to have more control in our creation of the MLP.